# The Transaction class
## Main components of a transaction
A transaction constitutes the fundamental component of a quantitative trading architecture. A transaction can be defined as an object with the following attributes:
1. Date
2. Ticker symbol
3. Number of shares
4. Total transaction amount
5. Transaction type
6. Description

## Additional implications
The following considerations will be discussed later:
1. The per share amount in a transaction can be derived from the total transaction amount and the number of shares
2. A ticker symbol does not always have to be associated with a transaction. This behavior is present in a bank transfer of cash into or out of a brokerage account

## Defining the transaction class simply
Assume first that the initialized transaction contains no actual information. This is a defensive coding practice that motivates the use of the None as the default value for all attributes. The datetime.date class is needed, so the datetime library is imported. The expected types are identified via the function annotations in the definition line

In [1]:
import datetime
class Transaction:
    """Represents a bank transfer, buy, dividends, fee, or sell"""
    def __init__(self, date: datetime.date = None, symbol: str = None,
                 num_shares: int = None, total_amount: float = None,
                 transact_type: str = None, description: str = None):
        self.date = date
        self.symbol = symbol
        self.num_shares = num_shares
        self.total_amount = total_amount
        self.transact_type = transact_type
        self.description = description

# Several demonstrative transactions
Consider the following theoretical transactions:
1. The purchase of 20 shares of Apple stock at a total price of \\$4000, placed on October 10th of 2018. Note that a price of \\$200 per share can be calculated from this information.
2. A bank transfer of \\$500 into a brokerage account on June 9th of 2019. Note that this transaction does not include a ticker symbol and as such the None keyword is used to intialize the bank transfer transaction object.

In [9]:
t0 = Transaction(datetime.date(2018, 10, 12),
                 'AAPL', 20, 4000, 'Buy', "Apple market buy")
t1 = Transaction(datetime.date(2019, 6, 9),
                 None, None, 500, "Bank transfer",
                 "Transfer from Chase")

In [11]:
import pandas as pd
labels = ['date', 'symbol','num_shares', 'total_amount',
          'transact_type', 'description']
df = pd.DataFrame([(t0.date, t0.symbol, t0.num_shares,
                    t0.total_amount, t0.transact_type, t0.description),
                   (t1.date, t1.symbol, t1.num_shares,
                    t1.total_amount, t1.transact_type, t1.description)],
                  columns=labels)
                    
df

,date,symbol,num_shares,total_amount,transact_type,description
0,2018-10-12,AAPL,20.0,4000,Buy,Apple market buy
1,2019-06-09,None,NaN,500,Bank transfer,Transfer from Chase


# Testing the basic transaction class
Parametrized pytest markers can be chained to test the proper assignment of attributes as follows:

In [ ]:
@pytest.mark.parametrize('date',
     [datetime.date(2018, 10, 12), datetime.date(1, 1, 1)])
@pytest.mark.parametrize('symbol', ['AAPL', None, 'BRK.A'])
@pytest.mark.parametrize('num_shares', [20, None, 0])
@pytest.mark.parametrize('total_amount', [0, None, 500, 300.46])
@pytest.mark.parametrize('transact_type', ['Buy', 'Fees', None])
@pytest.mark.parametrize('description', [None, '', "Market buy"])
def test_init(date, symbol, num_shares, total_amount, transact_type,
              description):
    t = Transaction(
            date, symbol, num_shares, total_amount, transact_type, description)
    assert \
        t.date == date and \
        t.symbol == symbol and \
        t.num_shares == num_shares and \
        t.total_amount == total_amount and \
        t.transact_type == transact_type and \
        t.description == description

However, this results in 648 tests, which takes a total of >1s to execute. It is not necessary to test every combination of attributes, however, so testing

`num_shares = 3` and `num_shares = 0`

independent of other atttributes should reduce the number of steps while ensuring proper coverage. [pytest.org test class documentation](https://docs.pytest.org/en/latest/getting-started.html#group-multiple-tests-in-a-class) informs the following restructuring:

In [ ]:
class TestInit(object):
    """Tests the initialization functions of the Transaction class"""

    @pytest.mark.parametrize(
        "in_var", [None, datetime.date(2018, 10, 12), datetime.date(1, 1, 1)])
    def test_date(self, in_var):
        """Verifies the date is correctly initialized"""
        t = Transaction(date=in_var)
        assert t.date == in_var

    @pytest.mark.parametrize(
        "in_var", [None, '', ' ', 'AAPL', 'BRK.A', 'AAPL', '^CRSPTMT'])
    def test_symbol(self, in_var):
        """Verifies the symbol is correctly initialized"""
        t = Transaction(symbol=in_var)
        assert t.symbol == in_var

    @pytest.mark.parametrize(
        "in_var", [None, 0, 1.2, 3, -1])
    def test_num_shares(self, in_var):
        """Verifies the number of shares is correctly initialized"""
        t = Transaction(num_shares=in_var)
        assert t.num_shares == in_var

Note that a negative number of shares or even a floating point value could be assigned to `num_shares`, based on the initial definition of the class, and these tests will still pass. This informs additional error-checking logic in the initialization of the `Transaction` class based on test-driven development principles.

# Re-defining the Transaction class
Flag that says inputs valid, and return None as the object if any inputs are passed that would make invalid
Basically an incorrect type will cause none to be returned, Invalid type is a non-date, etc,